In [ ]:
import findspark

findspark.init()

In [ ]:
import time

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

sc = SparkContext("local[2]", appName='Streaming-Test-2')

## 10 seconds for dividing into batches
ssc = StreamingContext(sc, 10)
sqlcontext =  SparkSession(sc)

host = 'localhost'
port = 3090

## Create an input from TCP source hostname:port. Data is received 
# ## using a TCP socket and receive byte is interpreted as UTF8 encoded \n delimited lines.
socket_stream = ssc.socketTextStream(host, port)
lines = socket_stream.window(20) ## perform transformation for 20 seconds window of each 10 seconds interval


def list_to_tup(x):
    item = x.split('::::')
    return (item[0], item[1], item[2])

## registers to a table
# lines.map(lambda x: list_to_tup(x)).foreachRDD(lambda rdd: rdd.toDF().registerTempTable('temporary'))
lines.map(list_to_tup).map(lambda x: (x[0], int(x[2]))).reduceByKey(lambda a, b: a + b).foreachRDD(lambda rdd: rdd.toDF().createOrReplaceTempView('temporary'))

Start `listener.py`

In [ ]:
ssc.start()

## wait for at least 1 min for temporary table to be created
time.sleep(60) 